In [ ]:
import os, re, io, json, time, base64, hashlib
from dataclasses import dataclass, asdict
from typing import Dict, Any, List, Tuple

import numpy as np
# === 兼容 SHAP 对 np.bool 的老引用 ===
if not hasattr(np, "bool"):
    np.bool = np.bool_  # type: ignore

import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import joblib
import shap
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch

from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATHS = [
    "/content/drive/My Drive/TEE_Dataset/attack_power_measurements.csv",
    "/content/drive/My Drive/TEE_Dataset/benign_power_measurements.csv",]
ART_DIR = "/content/drive/My Drive/TEE_Dataset"  # 产物输出目录（图、日志、模型等）
os.makedirs(ART_DIR, exist_ok=True)

In [ ]:
# 规范化后的“电力测量”特征名（尽量覆盖 SDN-MG25）
CANON = ["Vdc", "Vin", "Iin", "P", "Q", "freq", "temp"]
SYNONYMS = {
    "Vdc":  [r"vdc", r"dc[-_\s]?bus", r"edc", r"dc\s*voltage", r"bus\s*voltage", r"edc_v"],
    "Vin":  [r"vin", r"input\s*voltage", r"ein", r"vin_v"],
    "Iin":  [r"iin", r"input\s*current", r"iin_a"],
    "P":    [r"(^|[^a-z])p([^a-z]|$)", r"active\s*power", r"pout", r"p_out", r"pin", r"p\(active\)", r"pin_w"],
    "Q":    [r"(^|[^a-z])q([^a-z]|$)", r"reactive\s*power", r"qout", r"qvar", r"q\(reactive\)", r"q_var"],
    "freq": [r"freq", r"frequency", r"hz"],
    "temp": [r"temp", r"temperature"],
}
# 滑窗与判决阈值（根据你的真实基线可调整）
W, S = 128, 64
TH_REVIEW, TH_BLOCK = 0.7, 1.2

In [ ]:
# -------------------------------
# 1) 读取 CSV 并映射特征名
# -------------------------------
def read_csv_safely(path: str) -> pd.DataFrame:
    for enc in ["utf-8", "utf-8-sig", "gb18030", "latin-1"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    return pd.read_csv(path)

def glob_paths(paths: List[str]) -> List[str]:
    found = []
    import glob
    for p in paths:
        if os.path.isdir(p):
            found += glob.glob(os.path.join(p, "*.csv"))
        elif "*" in p or "?" in p or "[" in p:
            found += glob.glob(p)
        else:
            found.append(p)
    # 去重且存在性过滤
    uniq = []
    for p in found:
        if os.path.exists(p) and p not in uniq:
            uniq.append(p)
    return uniq

def load_power_measurements(paths: List[str]) -> pd.DataFrame:
    files = glob_paths(paths)
    frames = []
    for p in files:
        df = read_csv_safely(p)
        df["__source__"] = os.path.basename(p)
        frames.append(df)
    if not frames:
        raise FileNotFoundError("未找到任何有效 CSV，请检查 PATHS。")
    return pd.concat(frames, ignore_index=True)

raw_df = load_power_measurements(PATHS)

def find_column(columns: List[str], patterns: List[str]) -> str:
    lower = {c: c.lower().strip() for c in columns}
    for c, lc in lower.items():
        for pat in patterns:
            if re.search(pat, lc):
                return c
    return ""

def map_features(df: pd.DataFrame) -> Dict[str, str]:
    mapping = {}
    cols = df.columns.tolist()
    for k in CANON:
        c = find_column(cols, SYNONYMS[k])
        if c:
            mapping[k] = c
    return mapping

In [ ]:
feat_map = map_features(raw_df)
# 至少需要四个特征，且必须包含 P
if len(feat_map) < 4 or "P" not in feat_map:
    raise RuntimeError(f"可识别特征不足（至少含 P 和其他 3 个）；当前映射: {feat_map}")

In [ ]:
def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == object:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

num_df = coerce_numeric(raw_df)

In [ ]:
# 用“识别到的”特征子集，重命名为规范名
col_rename = {feat_map[k]: k for k in feat_map}
used_feats = list(feat_map.keys())
model_df = num_df[[feat_map[k] for k in used_feats]].rename(columns=col_rename).dropna().copy()

In [ ]:
# 预测下一步 P（one-step-ahead）
model_df["P_next"] = model_df["P"].shift(-1)
model_df = model_df.dropna().reset_index(drop=True)

In [ ]:
# 按时间顺序切分
n = len(model_df)
split = int(n * 0.7)
train_df = model_df.iloc[:split].copy()
test_df  = model_df.iloc[split:].copy()

X_train = train_df[used_feats].copy().fillna(method="ffill").fillna(method="bfill")
y_train = train_df["P_next"].values
X_test  = test_df[used_feats].copy().fillna(method="ffill").fillna(method="bfill")
y_test  = test_df["P_next"].values

/tmp/ipython-input-1902179850.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_train = train_df[used_feats].copy().fillna(method="ffill").fillna(method="bfill")
/tmp/ipython-input-1902179850.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test  = test_df[used_feats].copy().fillna(method="ffill").fillna(method="bfill")


In [ ]:
# -------------------------------
# 2) 建模（树模型便于高效 SHAP）
# -------------------------------
gbr = GradientBoostingRegressor(random_state=0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
mse_test = float(mean_squared_error(y_test, y_pred))

In [ ]:
# 保存中间件
joblib.dump(gbr, os.path.join(ART_DIR, "model_gbr.joblib"))
X_train.to_csv(os.path.join(ART_DIR, "X_train.csv"), index=False)
X_test.to_csv(os.path.join(ART_DIR, "X_test.csv"), index=False)
pd.DataFrame({"y_train": y_train}).to_csv(os.path.join(ART_DIR, "y_train.csv"), index=False)
pd.DataFrame({"y_test": y_test}).to_csv(os.path.join(ART_DIR, "y_test.csv"), index=False)
with open(os.path.join(ART_DIR, "feature_names.json"), "w") as f:
    json.dump(used_feats, f, ensure_ascii=False)
with open(os.path.join(ART_DIR, "feat_map.json"), "w") as f:
    json.dump(feat_map, f, ensure_ascii=False)

In [ ]:
print("识别到的列映射:", feat_map)
print("使用特征:", used_feats)
print("TEST MSE:", mse_test)

识别到的列映射: {'Vdc': 'EDC_V', 'Vin': 'Vin_V', 'Iin': 'Iin_A', 'P': 'Pin_W', 'Q': 'Q_var'}
使用特征: ['Vdc', 'Vin', 'Iin', 'P', 'Q']
TEST MSE: 5814.664261573221


In [ ]:
# -------------------------------
# 3) Enclave（仿真版）+ SHAP
# -------------------------------
@dataclass
class AttestationQuote:
    mr_enclave: str
    nonce: str
    timestamp: float

In [ ]:
@dataclass
class Evidence:
    ts: float
    batch_id: int
    input_hash: str
    model_hash: str
    enclave_hash: str
    mse: float
    shap_topk: Dict[str, float]
    shap_mean_abs: Dict[str, float]
    drift_score: float
    decision: str
    quote: Dict[str, Any]

In [ ]:
class AppendOnlyLog:
    """ 简单链式只增日志（每条包含前条 hash）。"""
    def __init__(self, path: str):
        self.path = path
        os.makedirs(os.path.dirname(path), exist_ok=True)
        if not os.path.exists(path):
            with open(path, "w") as f:
                f.write("index,prev_hash,record_hash,record_json\n")
        self._last_hash = "GENESIS"

    def _hash(self, b: bytes) -> str:
        return hashlib.sha256(b).hexdigest()

    def append(self, obj: Dict[str, Any]):
        payload = json.dumps(obj, sort_keys=True).encode()
        record_hash = self._hash(payload + self._last_hash.encode())
        with open(self.path, "a") as f:
            index = sum(1 for _ in open(self.path)) - 1
            row = f"{index},{self._last_hash},{record_hash},{json.dumps(obj, separators=(',',':'))}\n"
            f.write(row)
        self._last_hash = record_hash

In [ ]:
class Enclave:
    """
    Enclave 仿真：
    - attest(): 生成 quote；真实环境应由控制端验证 mr_enclave + nonce
    - 内部 SHAP：TreeExplainer 在域内解释；只输出摘要
    - 漂移检测：当前 mean|SHAP| 与基线 mean|SHAP| 的 L2 距离
    - 证据：哈希、指标、归因摘要、quote，Ed25519 签名
    """
    def __init__(self, model, feature_names: List[str], baseline_X: pd.DataFrame):
        self.model = model
        self.feature_names = feature_names
        self._mr_enclave = hashlib.sha256(repr(self.model).encode()).hexdigest()
        self._nonce = None
        self._sk = Ed25519PrivateKey.generate()
        self._pk = self._sk.public_key()
        self.explainer = shap.TreeExplainer(self.model)
        self.baseline_shap = np.abs(self.explainer.shap_values(baseline_X)).mean(axis=0)

    def attest(self) -> AttestationQuote:
        self._nonce = base64.urlsafe_b64encode(os.urandom(16)).decode()
        return AttestationQuote(mr_enclave=self._mr_enclave, nonce=self._nonce, timestamp=time.time())

    def infer_with_xai(self, X: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        yhat = self.model.predict(X)
        shap_vals = self.explainer.shap_values(X)
        return yhat, shap_vals

    def drift_score(self, shap_vals: np.ndarray) -> float:
        cur = np.abs(shap_vals).mean(axis=0)
        return float(np.linalg.norm(cur - self.baseline_shap, ord=2))

    def sign(self, data: bytes) -> str:
        sig = self._sk.sign(data)
        return base64.b64encode(sig).decode()

    def public_key_pem(self) -> str:
        return self._pk.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo).decode()


In [ ]:
def hash_dataframe(df: pd.DataFrame) -> str:
    buf = io.BytesIO()
    np.save(buf, df.values, allow_pickle=False)
    return hashlib.sha256(buf.getvalue()).hexdigest()

def build_evidence(enc: Enclave, batch_id: int, Xb: pd.DataFrame, yb: np.ndarray, yhat: np.ndarray, shap_vals: np.ndarray) -> Evidence:
    mse = float(mean_squared_error(yb, yhat))
    drift = enc.drift_score(shap_vals)
    mean_abs = np.abs(shap_vals).mean(axis=0)
    order = np.argsort(-mean_abs)[:6]
    topk = {enc.feature_names[i]: float(mean_abs[i]) for i in order}
    mean_map = {enc.feature_names[i]: float(mean_abs[i]) for i in range(len(mean_abs))}
    decision = "block" if drift >= TH_BLOCK else ("review" if drift >= TH_REVIEW else "allow")
    ev = Evidence(
        ts=time.time(),
        batch_id=batch_id,
        input_hash=hash_dataframe(Xb),
        model_hash=enc._mr_enclave,  # 此处用 enclave 哈希作为模型度量替身
        enclave_hash=enc._mr_enclave,
        mse=mse,
        shap_topk=topk,
        shap_mean_abs=mean_map,
        drift_score=drift,
        decision=decision,
        quote=asdict(enc.attest()),
    )
    return ev

In [ ]:
# 基线样本（建议换成“确认正常段”）
base_slice = X_train.iloc[: min(512, len(X_train))].copy()
enclave = Enclave(gbr, used_feats, base_slice)


In [ ]:
log_path = os.path.join(ART_DIR, "evidence_log.csv")
log = AppendOnlyLog(log_path)

In [ ]:
drift_series: List[float] = []
decisions: List[str] = []
worst_drift = -1.0
ev_first = None
ev_worst = None
shap_worst = None
X_worst = None


In [ ]:
for start in range(0, len(X_test) - W + 1, S):
    end = start + W
    Xb = X_test.iloc[start:end].copy()
    yb = y_test[start:end].copy()
    yhat_b, shap_b = enclave.infer_with_xai(Xb)
    ev = build_evidence(enclave, start//S, Xb, yb, yhat_b, shap_b)

    payload = json.dumps(asdict(ev), sort_keys=True).encode()
    sig = enclave.sign(payload)
    rec = asdict(ev) | {"signature": sig, "pubkey": enclave.public_key_pem()}
    log.append(rec)

    drift_series.append(ev.drift_score)
    decisions.append(ev.decision)
    if ev_first is None:
        ev_first = ev
    if ev.drift_score > worst_drift:
        worst_drift = ev.drift_score
        ev_worst = ev
        shap_worst = shap_b
        X_worst = Xb.copy()


In [ ]:
# -------------------------------
# 4) 可视化（要求：不用 seaborn；单图单轴；不指定颜色）
# -------------------------------
# 归因漂移曲线
plt.figure(figsize=(9,4.2))
plt.plot(np.arange(len(drift_series)), drift_series)
plt.axhline(TH_REVIEW, linestyle="--")
plt.axhline(TH_BLOCK, linestyle="--")
plt.xlabel("window index")
plt.ylabel("drift score (||mean|SHAP|-baseline||)")
plt.title("SDN-MG25 Power Measurements: Attribution Drift over Time")
plt.tight_layout()
drift_plot_path = os.path.join(ART_DIR, "drift_curve.png")
plt.savefig(drift_plot_path, dpi=170, bbox_inches="tight")
plt.close()

In [ ]:
# 基线 vs 最差窗口 Top-k 归因对比
mean_base = enclave.baseline_shap
mean_worst = np.abs(shap_worst).mean(axis=0)
k = min(8, len(used_feats))
idx = np.argsort(-mean_worst)[:k]
labels = [used_feats[i] for i in idx]
x = np.arange(k)


In [ ]:
plt.figure(figsize=(8,4.5))
plt.bar(x-0.15, mean_base[idx], width=0.3, label="baseline")
plt.bar(x+0.15, mean_worst[idx], width=0.3, label="worst-window")
plt.xticks(x, labels, rotation=30)
plt.ylabel("mean |SHAP|")
plt.title(f"Top-{k} Attribution Shift (Baseline vs Worst)")
plt.legend()
plt.tight_layout()
shift_plot_path = os.path.join(ART_DIR, "topk_shap_shift.png")
plt.savefig(shift_plot_path, dpi=170, bbox_inches="tight")
plt.close()

In [ ]:
# -------------------------------
# 5) 系统架构图（英文标签避免字体问题）
# -------------------------------
fig, ax = plt.subplots(figsize=(12,8))
ax.set_xlim(0,14); ax.set_ylim(0,10); ax.axis('off')

def add_box(x, y, w, h, label, lw=1.5):
    rect = Rectangle((x,y), w, h, linewidth=lw, edgecolor='black', facecolor='none')
    ax.add_patch(rect)
    ax.text(x + w/2, y + h/2, label, ha='center', va='center', fontsize=10, wrap=True)
    return rect

def add_arrow(xy1, xy2, connectionstyle="arc3,rad=0.0", lw=1.2):
    arrow = FancyArrowPatch(xy1, xy2, arrowstyle='->', mutation_scale=12, lw=lw,
                            connectionstyle=connectionstyle, color='black')
    ax.add_patch(arrow)

# 左侧数据源
add_box(0.5, 7.5, 3.0, 1.0, "SDN-MG25\nPower Measurements\n(" + ", ".join(used_feats) + ")")
add_box(0.5, 5.8, 3.0, 1.0, "Telemetry & Logs\n(Network, Syscalls)")
add_box(0.5, 4.1, 3.0, 1.0, "Apps / Dispatch / HMI")

# 控制服务
add_box(6.8, 9.2, 3.0, 0.8, "Remote Attestation")
add_box(10.2, 9.2, 3.0, 0.8, "Key Management")

# 安全入口
add_box(4.0, 9.2, 2.6, 0.8, "Secure Ingress\n(Attested Channel)")

# Enclave 容器
add_box(4.0, 2.5, 6.6, 6.5, "TEE Enclave", lw=2.0)

# Enclave 内部
add_box(4.5, 7.6, 5.6, 1.1, "Model Runtime (GBR Forecast)\n[Sealed Model]")
add_box(4.5, 6.1, 5.6, 1.0, "XAI Attributor (in-enclave)\nSHAP TreeExplainer")
add_box(4.5, 4.7, 5.6, 1.0, "Attribution Drift Detector\n(Baseline mean|SHAP|)")
add_box(4.5, 3.4, 5.6, 0.9, "Evidence Builder (signed)\n(hashes, metrics, SHAP summary, quote)")

# 右侧消费者
add_box(11.2, 6.2, 2.3, 1.3, "Ops Dashboard &\nPolicy Engine")
add_box(11.2, 4.2, 2.3, 1.3, "Append-only\nEvidence Log")

# 底部输出
add_box(5.2, 0.5, 3.0, 1.0, "SCADA/EMS\n(Actuation/HMI)")

# 箭头
add_arrow((3.5, 8.0), (4.0, 9.0), "arc3,rad=0.2")
add_arrow((3.5, 6.3), (4.0, 9.0), "arc3,rad=0.1")
add_arrow((3.5, 4.6), (4.0, 9.0), "arc3,rad=0.0")

add_arrow((8.3, 9.2), (6.6, 9.2), "arc3,rad=0.0")
add_arrow((11.7, 9.2), (9.8, 9.2), "arc3,rad=0.0")
add_arrow((8.3, 9.2), (7.3, 8.5), "arc3,rad=-0.3")
add_arrow((11.7, 9.2), (10.6, 8.5), "arc3,rad=-0.3")

add_arrow((5.3, 9.2), (5.3, 8.9))
add_arrow((7.3, 7.6), (7.3, 7.2))
add_arrow((7.3, 6.1), (7.3, 5.8))
add_arrow((7.3, 4.7), (7.3, 4.4))

add_arrow((10.1, 3.85), (11.2, 5.0), "arc3,rad=0.2")
add_arrow((10.1, 5.2), (11.2, 6.8), "arc3,rad=0.2")
add_arrow((12.35, 6.8), (6.6, 9.0), "arc3,rad=0.3")

ax.text(0.5, 9.7, "TEE + XAI for Secure Inference on SDN-MG25 Power Measurements", fontsize=12, ha='left', va='center')
arch_path = os.path.join(ART_DIR, "tee_xai_arch_sdnmg25.png")
plt.tight_layout()
plt.savefig(arch_path, dpi=200, bbox_inches='tight')
plt.close()

In [ ]:

# -------------------------------
# 6) 汇总输出
# -------------------------------
summary = {
    "rows_total": int(len(raw_df)),
    "mapped_features": feat_map,
    "used_features": used_feats,
    "train_rows": int(len(X_train)),
    "test_rows": int(len(X_test)),
    "test_mse": mse_test,
    "worst_drift": float(worst_drift),
    "evidence_log": log_path,
    "drift_curve_png": drift_plot_path,
    "topk_shift_png": shift_plot_path,
    "architecture_png": arch_path,
    "artifacts_dir": ART_DIR
}
print(json.dumps(summary, indent=2, ensure_ascii=False))

{
  "rows_total": 5400,
  "mapped_features": {
    "Vdc": "EDC_V",
    "Vin": "Vin_V",
    "Iin": "Iin_A",
    "P": "Pin_W",
    "Q": "Q_var"
  },
  "used_features": [
    "Vdc",
    "Vin",
    "Iin",
    "P",
    "Q"
  ],
  "train_rows": 3779,
  "test_rows": 1620,
  "test_mse": 5814.664261573221,
  "worst_drift": 591.9383730442659,
  "evidence_log": "/content/drive/My Drive/TEE_Dataset/evidence_log.csv",
  "drift_curve_png": "/content/drive/My Drive/TEE_Dataset/drift_curve.png",
  "topk_shift_png": "/content/drive/My Drive/TEE_Dataset/topk_shap_shift.png",
  "architecture_png": "/content/drive/My Drive/TEE_Dataset/tee_xai_arch_sdnmg25.png",
  "artifacts_dir": "/content/drive/My Drive/TEE_Dataset"
}


In [ ]:
# ===== Evaluation add-on (paste into a new Colab cell) =====
import os, json, io, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ---- change if your artifacts dir differs ----

# Load artifacts
X_train = pd.read_csv(f"{ART_DIR}/X_train.csv")
X_test  = pd.read_csv(f"{ART_DIR}/X_test.csv")
y_train = pd.read_csv(f"{ART_DIR}/y_train.csv")["y_train"].values
y_test  = pd.read_csv(f"{ART_DIR}/y_test.csv")["y_test"].values
import joblib, shap
model = joblib.load(f"{ART_DIR}/model_gbr.joblib")
with open(f"{ART_DIR}/feature_names.json","r") as f:
    feature_names = json.load(f)


In [ ]:
# Predictions for test set
y_pred = model.predict(X_test)

In [ ]:
# ---- 2.1 基础误差指标 ----
rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
mae  = float(mean_absolute_error(y_test, y_pred))
r2   = float(r2_score(y_test, y_pred))
mape = float(np.mean(np.abs((y_test - y_pred) / np.maximum(1e-6, np.abs(y_test)))) * 100.0)
print({"RMSE":rmse, "MAE":mae, "R2":r2, "MAPE(%)":mape})

{'RMSE': 76.2539458754314, 'MAE': 59.69263311489408, 'R2': 0.9896598113426417, 'MAPE(%)': 0.9079901877336461}


In [ ]:
# 残差分布
res = y_test - y_pred
plt.figure(figsize=(7,4))
plt.hist(res, bins=40)
plt.title("Residual Histogram (test)")
plt.xlabel("residual")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(f"{ART_DIR}/residual_hist.png", dpi=160, bbox_inches="tight"); plt.close()


In [ ]:
# 残差随时间
plt.figure(figsize=(9,3.8))
plt.plot(np.arange(len(res)), res)
plt.title("Residuals over time (test)")
plt.xlabel("t")
plt.ylabel("residual")
plt.tight_layout()
plt.savefig(f"{ART_DIR}/residual_over_time.png", dpi=160, bbox_inches="tight"); plt.close()

In [ ]:
# ---- 2.2 XAI 稳定性指标（窗口级）----
W, S = 128, 64  # 与主脚本一致
explainer = shap.TreeExplainer(model)


In [ ]:
# baseline: 用训练集前 512 行（与主脚本一致；更建议换成“确认正常段”）
base = X_train.iloc[:min(512, len(X_train))].copy()
shap_base = np.abs(explainer.shap_values(base)).mean(axis=0)
p_base = shap_base / np.maximum(1e-12, shap_base.sum())  # 归一化分布

In [ ]:
def spearman(a,b):
    # 以秩相关近似（越大越相似）
    ra = pd.Series(a).rank(method="average").values
    rb = pd.Series(b).rank(method="average").values
    cov = np.cov(ra, rb)[0,1]
    return float(cov / (np.std(ra)*np.std(rb) + 1e-12))

def jaccard_topk(a, b, k=5):
    ia = np.argsort(-a)[:k]; ib = np.argsort(-b)[:k]
    A, B = set(ia.tolist()), set(ib.tolist())
    return float(len(A & B) / max(1, len(A | B)))

def psi(p, q, eps=1e-12, bins=10):
    # 对“归一化 |SHAP| 分布”用等宽分箱（按特征维度划分）
    # 若特征较少，改用直接PSI定义（p_i, q_i）
    p = p / (p.sum()+eps); q = q / (q.sum()+eps)
    # 直接 PSI（不再分箱）：sum( (p_i - q_i) * ln(p_i/q_i) )
    v = (p - q) * np.log((p + eps) / (q + eps))
    return float(np.sum(v))

def js_div(p, q, eps=1e-12):
    p = p / (p.sum()+eps); q = q / (q.sum()+eps)
    m = 0.5*(p+q)
    kl = lambda x,y: np.sum(x*np.log((x+eps)/(y+eps)))
    return float(0.5*kl(p,m) + 0.5*kl(q,m))

In [ ]:
rows = []
for start in range(0, len(X_test) - W + 1, S):
    end = start + W
    Xw = X_test.iloc[start:end].copy()
    yw = y_test[start:end]
    yhat = model.predict(Xw)
    mse_w = float(mean_squared_error(yw, yhat))
    shap_w = np.abs(explainer.shap_values(Xw)).mean(axis=0)
    p_w = shap_w / np.maximum(1e-12, shap_w.sum())

    drift = float(np.linalg.norm(shap_w - shap_base))  # 与主脚本一致
    sp = spearman(shap_w, shap_base)
    jac = jaccard_topk(shap_w, shap_base, k=min(5, len(feature_names)))
    psi_v = psi(p_base, p_w)
    js_v = js_div(p_base, p_w)

    rows.append({
        "window_idx": start//S,
        "mse": mse_w,
        "drift_l2": drift,
        "spearman": sp,
        "topk_jaccard": jac,
        "psi": psi_v,
        "js": js_v
    })

In [ ]:
win_df = pd.DataFrame(rows)
win_df.to_csv(f"{ART_DIR}/window_eval.csv", index=False)
print("Saved metrics to:", f"{ART_DIR}/window_eval.csv")
print(win_df.describe().round(4).to_dict())

Saved metrics to: /content/drive/My Drive/TEE_Dataset/window_eval.csv
{'window_idx': {'count': 24.0, 'mean': 11.5, 'std': 7.0711, 'min': 0.0, '25%': 5.75, '50%': 11.5, '75%': 17.25, 'max': 23.0}, 'mse': {'count': 24.0, 'mean': 5655.1191, 'std': 1436.9871, 'min': 2901.582, '25%': 5030.2563, '50%': 5245.7706, '75%': 6793.1752, 'max': 7894.6389}, 'drift_l2': {'count': 24.0, 'mean': 279.9726, 'std': 194.5924, 'min': 10.798, '25%': 132.4116, '50%': 242.8536, '75%': 460.3039, 'max': 591.9384}, 'spearman': {'count': 24.0, 'mean': 1.1042, 'std': 0.1593, 'min': 0.875, '25%': 0.875, '50%': 1.125, '75%': 1.25, 'max': 1.25}, 'topk_jaccard': {'count': 24.0, 'mean': 1.0, 'std': 0.0, 'min': 1.0, '25%': 1.0, '50%': 1.0, '75%': 1.0, 'max': 1.0}, 'psi': {'count': 24.0, 'mean': 0.0437, 'std': 0.0294, 'min': 0.0066, '25%': 0.0188, '50%': 0.038, '75%': 0.0681, 'max': 0.1103}, 'js': {'count': 24.0, 'mean': 0.0052, 'std': 0.0035, 'min': 0.0008, '25%': 0.0023, '50%': 0.0046, '75%': 0.0079, 'max': 0.0135}}


In [ ]:
# 额外：输出全局（test）特征重要度
g_shap = np.abs(explainer.shap_values(X_test.sample(min(512, len(X_test)), random_state=0))).mean(axis=0)
pd.DataFrame({"feature": feature_names, "mean_abs_shap": g_shap}).sort_values("mean_abs_shap", ascending=False)\
  .to_csv(f"{ART_DIR}/global_importance.csv", index=False)
print("Saved global SHAP importance to:", f"{ART_DIR}/global_importance.csv")

Saved global SHAP importance to: /content/drive/My Drive/TEE_Dataset/global_importance.csv


In [ ]:
# ===== Evaluation add-on (paste into a new Colab cell) =====
import os, json, io, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

ART_DIR = "/content/drive/My Drive/TEE_Dataset"  # 产物输出目录（图、日志、模型等）

# Load artifacts
X_train = pd.read_csv(f"{ART_DIR}/X_train.csv")
X_test  = pd.read_csv(f"{ART_DIR}/X_test.csv")
y_train = pd.read_csv(f"{ART_DIR}/y_train.csv")["y_train"].values
y_test  = pd.read_csv(f"{ART_DIR}/y_test.csv")["y_test"].values
import joblib, shap
model = joblib.load(f"{ART_DIR}/model_gbr.joblib")
with open(f"{ART_DIR}/feature_names.json","r") as f:
    feature_names = json.load(f)

# Predictions for test set
y_pred = model.predict(X_test)

# ---- 2.1 基础误差指标 ----
rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
mae  = float(mean_absolute_error(y_test, y_pred))
r2   = float(r2_score(y_test, y_pred))
mape = float(np.mean(np.abs((y_test - y_pred) / np.maximum(1e-6, np.abs(y_test)))) * 100.0)
print({"RMSE":rmse, "MAE":mae, "R2":r2, "MAPE(%)":mape})

# 残差分布
res = y_test - y_pred
plt.figure(figsize=(7,4))
plt.hist(res, bins=40)
plt.title("Residual Histogram (test)")
plt.xlabel("residual")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(f"{ART_DIR}/residual_hist.png", dpi=160, bbox_inches="tight"); plt.close()

# 残差随时间
plt.figure(figsize=(9,3.8))
plt.plot(np.arange(len(res)), res)
plt.title("Residuals over time (test)")
plt.xlabel("t")
plt.ylabel("residual")
plt.tight_layout()
plt.savefig(f"{ART_DIR}/residual_over_time.png", dpi=160, bbox_inches="tight"); plt.close()

# ---- 2.2 XAI 稳定性指标（窗口级）----
W, S = 128, 64  # 与主脚本一致
explainer = shap.TreeExplainer(model)

# baseline: 用训练集前 512 行（与主脚本一致；更建议换成“确认正常段”）
base = X_train.iloc[:min(512, len(X_train))].copy()
shap_base = np.abs(explainer.shap_values(base)).mean(axis=0)
p_base = shap_base / np.maximum(1e-12, shap_base.sum())  # 归一化分布

def spearman(a,b):
    # 以秩相关近似（越大越相似）
    ra = pd.Series(a).rank(method="average").values
    rb = pd.Series(b).rank(method="average").values
    cov = np.cov(ra, rb)[0,1]
    return float(cov / (np.std(ra)*np.std(rb) + 1e-12))

def jaccard_topk(a, b, k=5):
    ia = np.argsort(-a)[:k]; ib = np.argsort(-b)[:k]
    A, B = set(ia.tolist()), set(ib.tolist())
    return float(len(A & B) / max(1, len(A | B)))

def psi(p, q, eps=1e-12, bins=10):
    # 对“归一化 |SHAP| 分布”用等宽分箱（按特征维度划分）
    # 若特征较少，改用直接PSI定义（p_i, q_i）
    p = p / (p.sum()+eps); q = q / (q.sum()+eps)
    # 直接 PSI（不再分箱）：sum( (p_i - q_i) * ln(p_i/q_i) )
    v = (p - q) * np.log((p + eps) / (q + eps))
    return float(np.sum(v))

def js_div(p, q, eps=1e-12):
    p = p / (p.sum()+eps); q = q / (q.sum()+eps)
    m = 0.5*(p+q)
    kl = lambda x,y: np.sum(x*np.log((x+eps)/(y+eps)))
    return float(0.5*kl(p,m) + 0.5*kl(q,m))

rows = []
for start in range(0, len(X_test) - W + 1, S):
    end = start + W
    Xw = X_test.iloc[start:end].copy()
    yw = y_test[start:end]
    yhat = model.predict(Xw)
    mse_w = float(mean_squared_error(yw, yhat))
    shap_w = np.abs(explainer.shap_values(Xw)).mean(axis=0)
    p_w = shap_w / np.maximum(1e-12, shap_w.sum())

    drift = float(np.linalg.norm(shap_w - shap_base))  # 与主脚本一致
    sp = spearman(shap_w, shap_base)
    jac = jaccard_topk(shap_w, shap_base, k=min(5, len(feature_names)))
    psi_v = psi(p_base, p_w)
    js_v = js_div(p_base, p_w)

    rows.append({
        "window_idx": start//S,
        "mse": mse_w,
        "drift_l2": drift,
        "spearman": sp,
        "topk_jaccard": jac,
        "psi": psi_v,
        "js": js_v
    })

win_df = pd.DataFrame(rows)
win_df.to_csv(f"{ART_DIR}/window_eval.csv", index=False)
print("Saved metrics to:", f"{ART_DIR}/window_eval.csv")
print(win_df.describe().round(4).to_dict())

# 额外：输出全局（test）特征重要度
g_shap = np.abs(explainer.shap_values(X_test.sample(min(512, len(X_test)), random_state=0))).mean(axis=0)
pd.DataFrame({"feature": feature_names, "mean_abs_shap": g_shap}).sort_values("mean_abs_shap", ascending=False)\
  .to_csv(f"{ART_DIR}/global_importance.csv", index=False)
print("Saved global SHAP importance to:", f"{ART_DIR}/global_importance.csv")


{'RMSE': 76.2539458754314, 'MAE': 59.69263311489408, 'R2': 0.9896598113426417, 'MAPE(%)': 0.9079901877336461}
Saved metrics to: /content/drive/My Drive/TEE_Dataset/window_eval.csv
{'window_idx': {'count': 24.0, 'mean': 11.5, 'std': 7.0711, 'min': 0.0, '25%': 5.75, '50%': 11.5, '75%': 17.25, 'max': 23.0}, 'mse': {'count': 24.0, 'mean': 5655.1191, 'std': 1436.9871, 'min': 2901.582, '25%': 5030.2563, '50%': 5245.7706, '75%': 6793.1752, 'max': 7894.6389}, 'drift_l2': {'count': 24.0, 'mean': 279.9726, 'std': 194.5924, 'min': 10.798, '25%': 132.4116, '50%': 242.8536, '75%': 460.3039, 'max': 591.9384}, 'spearman': {'count': 24.0, 'mean': 1.1042, 'std': 0.1593, 'min': 0.875, '25%': 0.875, '50%': 1.125, '75%': 1.25, 'max': 1.25}, 'topk_jaccard': {'count': 24.0, 'mean': 1.0, 'std': 0.0, 'min': 1.0, '25%': 1.0, '50%': 1.0, '75%': 1.0, 'max': 1.0}, 'psi': {'count': 24.0, 'mean': 0.0437, 'std': 0.0294, 'min': 0.0066, '25%': 0.0188, '50%': 0.038, '75%': 0.0681, 'max': 0.1103}, 'js': {'count': 24.0, 

In [ ]:
# ============================
# Add-on: LLM in TEE + SHAP plots
# ============================
# 先安装依赖（Colab 默认有 matplotlib；若缺少 transformers/sentencepiece 则解注运行）
# !pip install -q transformers accelerate sentencepiece shap

import os, json, io, time, base64, hashlib, numpy as np, pandas as pd, matplotlib.pyplot as plt
from dataclasses import dataclass, asdict
from typing import Dict, Any, List, Tuple

# 载入数据与模型
import joblib, shap
X_train = pd.read_csv(f"{ART_DIR}/X_train.csv")
X_test  = pd.read_csv(f"{ART_DIR}/X_test.csv")
y_train = pd.read_csv(f"{ART_DIR}/y_train.csv")["y_train"].values
y_test  = pd.read_csv(f"{ART_DIR}/y_test.csv")["y_test"].values
with open(f"{ART_DIR}/feature_names.json","r") as f:
    FEATURES = json.load(f)
model = joblib.load(f"{ART_DIR}/model_gbr.joblib")  # 如果你已换成自定义 wrapper，确保也能 joblib.load()

# -----------------------------
# 1) LLM 加入 Enclave（解释/建议）
# -----------------------------
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

LLM_NAME = "google/flan-t5-small"   # 轻量示例；可换 flan-t5-base（更强但更慢）
device = 0 if torch.cuda.is_available() else -1
tok = AutoTokenizer.from_pretrained(LLM_NAME)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(LLM_NAME)
llm_pipe = pipeline("text2text-generation", model=llm_model, tokenizer=tok, device=device)

# —— Enclave 内的 LLM 解释函数（基于 SHAP Top-K + 指标生成文本）——
def llm_explain(topk: Dict[str, float], drift: float, mse: float, decision: str) -> str:
    # 仅传入必要摘要，避免外泄中间量；真实 TEE 内也要限制 prompt 内容与长度
    ranked = ", ".join([f"{k}:{v:.2f}" for k,v in topk.items()])
    prompt = (
        "You are an operations assistant for a microgrid.\n"
        "Given SHAP attributions and risk metrics, write a concise, factual incident note:\n"
        f"- top_features(mean|SHAP|): {ranked}\n"
        f"- drift_score: {drift:.3f}\n"
        f"- mse: {mse:.3f}\n"
        f"- decision: {decision}\n"
        "Return: 1) risk_root_cause (50 words max); 2) likely_impacts; 3) recommended_actions.\n"
    )
    out = llm_pipe(prompt, max_new_tokens=120, temperature=0.1)[0]["generated_text"]
    return out.strip()

# —— 对接你主脚本的“窗口级证据”：如果你保存了 window_eval.csv 就用它；没有就按一个窗口重算 ——
win_eval_csv = os.path.join(ART_DIR, "window_eval.csv")
if os.path.exists(win_eval_csv):
    win_df = pd.read_csv(win_eval_csv)
    worst_idx = int(win_df.sort_values("drift_l2", ascending=False).iloc[0]["window_idx"])
    W, S = 128, 64
else:
    # 回落：和主脚本同样的窗口参数
    W, S = 128, 64
    worst_idx = 0

start = worst_idx * S
end   = start + W
Xw = X_test.iloc[start:end].copy()
yw = y_test[start:end].copy()

# -----------------------------
# 2) 计算 SHAP（自动选择 Explainer）
# -----------------------------
# 判别是否树模型（尽量鲁棒）
IS_TREE = model.__class__.__name__.lower().find("gradientboosting")>=0 \
          or model.__class__.__name__.lower().find("randomforest")>=0

if IS_TREE:
    explainer = shap.TreeExplainer(model)
    shap_vals = explainer.shap_values(Xw)
    expected_value = explainer.expected_value
else:
    # 通用：KernelExplainer（注意速度，抽样背景 + 限制 nsamples）
    bg = X_train.sample(min(64, len(X_train)), random_state=0)
    f = lambda data: model.predict(pd.DataFrame(data, columns=FEATURES))
    explainer = shap.KernelExplainer(f, bg.values)
    shap_vals = explainer.shap_values(Xw.values, nsamples=200)
    expected_value = explainer.expected_value

# 形成 Top-K（用于 LLM 解释）
mean_abs = np.abs(shap_vals).mean(axis=0)
order = np.argsort(-mean_abs)[:min(6, len(FEATURES))]
topk = {FEATURES[i]: float(mean_abs[i]) for i in order}

# 如果你有窗口 MSE / 决策，可读取；否则现算
from sklearn.metrics import mean_squared_error
yhat = model.predict(Xw)
mse_w = float(mean_squared_error(yw, yhat))
drift_w = float(np.linalg.norm(mean_abs - np.abs(explainer.shap_values(X_train.iloc[:min(512, len(X_train))])).mean(axis=0))) \
            if IS_TREE else float(np.linalg.norm(mean_abs - np.abs(explainer.shap_values(bg.values, nsamples=100)).mean(axis=0)))
decision = "block" if drift_w >= 1.2 else ("review" if drift_w >= 0.7 else "allow")

# -----------------------------
# 3) 在 TEE 中用 LLM 生成解释（并签名）
# -----------------------------
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat

sk = Ed25519PrivateKey.generate()
pk = sk.public_key()

llm_text = llm_explain(topk, drift_w, mse_w, decision)
sig = sk.sign(llm_text.encode())
sig_b64 = base64.b64encode(sig).decode()
pubkey_pem = pk.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo).decode()

with open(os.path.join(ART_DIR, "llm_summary.json"), "w") as f:
    json.dump({
        "window_idx": worst_idx,
        "topk": topk,
        "drift_score": drift_w,
        "mse": mse_w,
        "decision": decision,
        "llm_summary": llm_text,
        "signature": sig_b64,
        "pubkey": pubkey_pem
    }, f, ensure_ascii=False, indent=2)

print("LLM summary saved ->", os.path.join(ART_DIR, "llm_summary.json"))

# -----------------------------
# 4) SHAP 可视化并落盘（官方图形）
# -----------------------------
# (a) summary bar
plt.figure()
shap.summary_plot(shap_vals, Xw if IS_TREE else Xw.values, feature_names=FEATURES, plot_type="bar", show=False)
bar_path = os.path.join(ART_DIR, "shap_summary_bar.png")
plt.tight_layout(); plt.savefig(bar_path, dpi=180, bbox_inches="tight"); plt.close()

# (b) summary beeswarm
plt.figure()
shap.summary_plot(shap_vals, Xw if IS_TREE else Xw.values, feature_names=FEATURES, show=False)
beeswarm_path = os.path.join(ART_DIR, "shap_summary_beeswarm.png")
plt.tight_layout(); plt.savefig(beeswarm_path, dpi=180, bbox_inches="tight"); plt.close()

# (c) dependence（对前两大特征）
if len(order) >= 2:
    f1, f2 = FEATURES[order[0]], FEATURES[order[1]]
    plt.figure()
    shap.dependence_plot(f1, shap_vals, Xw if IS_TREE else Xw.values, feature_names=FEATURES, interaction_index=f2, show=False)
    dep_path = os.path.join(ART_DIR, f"shap_dependence_{f1}_x_{f2}.png")
    plt.tight_layout(); plt.savefig(dep_path, dpi=180, bbox_inches="tight"); plt.close()

# (d) waterfall（选一个代表样本）
# 需要 expected_value；构造 Explanation 对象以兼容新旧 SHAP API
i = int(np.argmax(np.abs(yw - yhat)))  # 取残差最大的点
try:
    exp = shap.Explanation(values=shap_vals[i],
                           base_values=expected_value,
                           data=(Xw.iloc[i].values if hasattr(Xw, "iloc") else Xw[i]),
                           feature_names=FEATURES)
    plt.figure()
    shap.plots.waterfall(exp, max_display=min(10, len(FEATURES)), show=False)
except Exception:
    # 旧版回退
    plt.figure()
    shap.waterfall_plot(expected_value, shap_vals[i], feature_names=FEATURES, show=False)
water_path = os.path.join(ART_DIR, "shap_waterfall_example.png")
plt.tight_layout(); plt.savefig(water_path, dpi=180, bbox_inches="tight"); plt.close()

print("SHAP plots saved ->", {
    "bar": bar_path,
    "beeswarm": beeswarm_path,
    "dependence": dep_path if len(order)>=2 else None,
    "waterfall": water_path
})


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


LLM summary saved -> /content/drive/My Drive/TEE_Dataset/llm_summary.json
SHAP plots saved -> {'bar': '/content/drive/My Drive/TEE_Dataset/shap_summary_bar.png', 'beeswarm': '/content/drive/My Drive/TEE_Dataset/shap_summary_beeswarm.png', 'dependence': '/content/drive/My Drive/TEE_Dataset/shap_dependence_P_x_Iin.png', 'waterfall': '/content/drive/My Drive/TEE_Dataset/shap_waterfall_example.png'}


<Figure size 640x480 with 0 Axes>